In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import time
import random
import pdb


### Make a list of the urls of the categories in the forum

In [3]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news',
 'https://curltalk.naturallycurly.com/categories/feedback',
 'https://curltalk.naturallycurly.com/categories/c2',
 'https://curltalk.naturallycurly.com/categories/c3',
 'https://curltalk.naturallycurly.com/categories/c4',
 'https://curltalk.naturallycurly.com/categories/general-discussion-about-curly-hair',
 'https://curltalk.naturallycurly.com/categories/salon-stylist-chatter',
 'https://curltalk.naturallycurly.com/categories/transitioning',
 'https://curltalk.naturallycurly.com/categories/thin-thinning-hair',
 'https://curltalk.naturallycurly.com/categories/readers-product-reviews',
 'https://curltalk.naturallycurly.com/categories/hall-of-shame-discussion',
 'https://curltalk.naturallycurly.com/categories/hall-of-fame-discussion'

#### Get the discussion topic urls for the first x pages of the category

In [181]:
# %%time

link_listdiscussion = []
for i in range(0, 150):
    url = f'{categories[42]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    #time.sleep(random.randint(20,45))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        #break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
#     time.sleep(random.randint(1,5))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/aussie-swap-board/p0
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p1
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p2
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p3
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p4
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p5
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p6
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p7
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p8
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p9
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p10
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p11
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p12
Error improper response code.

https://curltalk.naturallycurly.com/categories/aussie-swap-board/p78
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p79
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p80
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p81
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p82
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p83
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p84
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p85
Error improper response code. Code is 404
https://curltalk.naturallycurly.com/categories/aussie-swap-board/p86
Error improper response code. Code is 404
h

Error improper response code. Code is 404
5
Error improper response code. Code is 404
6
Error improper response code. Code is 404
7
Error improper response code. Code is 404
8
Error improper response code. Code is 404
9
Error improper response code. Code is 404
10
Error improper response code. Code is 404
11
Error improper response code. Code is 404
12
Error improper response code. Code is 404
13
Error improper response code. Code is 404
14
Error improper response code. Code is 404
15
Error improper response code. Code is 404
16
Error improper response code. Code is 404
17
Error improper response code. Code is 404
18
Error improper response code. Code is 404
19
Error improper response code. Code is 404
20
Error improper response code. Code is 404
21
Error improper response code. Code is 404
22
Error improper response code. Code is 404
23
Error improper response code. Code is 404
24
Error improper response code. Code is 404
25
Error improper response code. Code is 404
26
Error improper 

Error improper response code. Code is 404
186
Error improper response code. Code is 404
187
Error improper response code. Code is 404
188
Error improper response code. Code is 404
189
Error improper response code. Code is 404
190
Error improper response code. Code is 404
191
Error improper response code. Code is 404
192
Error improper response code. Code is 404
193
Error improper response code. Code is 404
194
Error improper response code. Code is 404
195
Error improper response code. Code is 404
196
Error improper response code. Code is 404
197
Error improper response code. Code is 404
198
Error improper response code. Code is 404
199
Error improper response code. Code is 404
200
Error improper response code. Code is 404
201
Error improper response code. Code is 404
202
Error improper response code. Code is 404
203
Error improper response code. Code is 404
204
Error improper response code. Code is 404
205
Error improper response code. Code is 404
206
Error improper response code. Code

In [182]:
len(list_for_mongo)

1356

### Create a mongoDB and have data stored from scrape 

In [183]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [184]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [185]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

461990


In [186]:
test_set = set(test)

In [187]:
# Number of unique entries so far
len(test_set)
#7675

7675